In [206]:
#!pip install pandas
#!pip install requests
#!pip install lxml
#!pip install pyarrow
#!pip install openpyxl

In [207]:
BASE_URL = 'https://www.transparencia.gob.pe/personal/pte_transparencia_personal_genera.aspx'

In [208]:
import pandas as pd
import requests
import urllib3
from io import StringIO
import re

In [209]:
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [220]:
def product_id_entidad():
    
    id_df = pd.read_excel('../data/id_ministerios.xlsx')

    id_months = list(range(1, 13))
    id_years = [
                2025
                , 2024
                
                #, 2023
                #, 2022
                #, 2021
                #, 2020
                #, 2019
                #, 2018
                #, 2017
                #, 2016
                #, 2015
                #, 2014   
                #, 2013
                #, 2012
                #, 2011
                #, 2010
           

                ]

    product_df = pd.MultiIndex.from_product(
        [id_df['id_entidad'], id_years, id_months],
        names=['id_entidad', 'year', 'month']
    ).to_frame(index=False)

    product_df = product_df.merge(id_df, on='id_entidad', how='left')
    return product_df


def list_data_files(folder, ext="parquet"):
    files = [f for f in os.listdir(folder) if f.endswith(f".{ext}")]
    data = []

    for file in files:
        parts = file.replace(f".{ext}", "").split("_")
        if len(parts) == 3 and all(p.isdigit() for p in parts):
            id_entidad, year, month = map(int, parts)
            data.append({
                "id_entidad": id_entidad,
                "year": year,
                "month": month,
                "file_name": file
            })

    return pd.DataFrame(data)

# List partitioned files in a folder structure like:
def list_data_files(base_folder):
    data = []

    for root, dirs, files in os.walk(base_folder):
        if "data.parquet" in files:
            match = re.search(r"id_entidad=(\d+)/year=(\d{4})/month=(\d{1,2})", root.replace("\\", "/"))
            if match:
                entity_id, year, month = match.groups()
                data.append({
                    "id_entidad": int(entity_id),
                    "year": int(year),
                    "month": int(month),
                    "file_name": os.path.join(root, "data.parquet")
                })

    # Always return a DataFrame with consistent columns
    return pd.DataFrame(data, columns=["id_entidad", "year", "month", "file_name"])

def control_downloaded_files(product_df, downloaded_df):

    control_df = product_df.merge(
        downloaded_df,
        on=["id_entidad", "year", "month"],
        how="left",
        indicator=True  # to show match status
    )
    # Add a flag column: found = True if file was present
    control_df["found"] = control_df["_merge"] == "both"

    # Optional: drop merge indicator column
    control_df.drop(columns="_merge", inplace=True)

    control_df
    return control_df

In [221]:
def url_to_df(base_url, id_entidad, year, month):
    params = {
        "id_entidad": id_entidad,
        "in_anno_consulta": year,
        "ch_mes_consulta": month,
        "ch_tipo_regimen": "",
        "vc_dni_funcionario": "",
        "vc_nombre_funcionario": "",
        "ch_tipo_descarga": 2,
    }

    try:
        headers = {"User-Agent": "Mozilla/5.0"}
        response = requests.get(base_url, params=params, headers=headers, verify=False)
        response.raise_for_status()

        # Wrap HTML string in StringIO to avoid FutureWarning
        html_io = StringIO(response.text)
        tables = pd.read_html(html_io, header=0)

        if tables:
            return tables[0]
        else:
            print("No tables found in HTML.")
            return None

    except Exception as e:
        print(f"Failed to get or parse HTML: {e}")
        return None

def save_df(df, entity_id, year, month, folder):
    import os

    filename = f"{entity_id}_{year}_{month}.parquet"
    filepath = os.path.join(folder, filename)

    df.to_parquet(filepath, index=False, engine="pyarrow")
    print(f"Saved to {filepath}")

# Function to save DataFrame to a partitioned folder structure
def save_df(df, entity_id, year, month, base_folder):
    import os

    # Partitioned folder path
    folder_path = os.path.join(
        base_folder,
        f"id_entidad={entity_id}",
        f"year={year}",
        f"month={month}"
    )

    os.makedirs(folder_path, exist_ok=True)

    filepath = os.path.join(folder_path, "data.parquet")
    df.to_parquet(filepath, index=False, engine="pyarrow")
    print(f"✅ Saved to {filepath}")



In [222]:
product_df = product_id_entidad()
downloaded_df = list_data_files('../data/bronze')
control_df = control_downloaded_files(product_df, downloaded_df)

control_df = control_df[control_df['found'] == False]
control_df

,id_entidad,year,month,sector,entidad,file_name,found
0,136,2025,1,AGRICULTURA,MIDAGRI,NaN,False
1,136,2025,2,AGRICULTURA,MIDAGRI,NaN,False
2,136,2025,3,AGRICULTURA,MIDAGRI,NaN,False
3,136,2025,4,AGRICULTURA,MIDAGRI,NaN,False
4,136,2025,5,AGRICULTURA,MIDAGRI,NaN,False
...,...,...,...,...,...,...,...
439,11476,2025,8,"VIVIENDA, CONSTRUCCION Y SANEAMIENTO",MVCS,NaN,False
440,11476,2025,9,"VIVIENDA, CONSTRUCCION Y SANEAMIENTO",MVCS,NaN,False
441,11476,2025,10,"VIVIENDA, CONSTRUCCION Y SANEAMIENTO",MVCS,NaN,False
442,11476,2025,11,"VIVIENDA, CONSTRUCCION Y SANEAMIENTO",MVCS,NaN,False


In [223]:
for _, row in control_df.iterrows():
    id_entidad = row['id_entidad']
    year = row['year']
    month = row['month']
    
    print(f"Downloading data for {id_entidad}, {year}, {month}...")

    df = url_to_df(BASE_URL, id_entidad, year, month)
    
    if df is not None and not df.empty:
        save_df(df, id_entidad, year, month, '../data/bronze')
    else:
        print(f"No data found for {id_entidad}, {year}, {month}.")

✅ Saved to ../data/bronze/id_entidad=136/year=2025/month=1/data.parquet
✅ Saved to ../data/bronze/id_entidad=136/year=2025/month=2/data.parquet
✅ Saved to ../data/bronze/id_entidad=136/year=2025/month=3/data.parquet
✅ Saved to ../data/bronze/id_entidad=136/year=2025/month=4/data.parquet
Failed to get or parse HTML: no text parsed from document (line 0)
No data found for 136, 2025, 5.
Failed to get or parse HTML: no text parsed from document (line 0)
No data found for 136, 2025, 6.
Failed to get or parse HTML: no text parsed from document (line 0)
No data found for 136, 2025, 7.
Failed to get or parse HTML: no text parsed from document (line 0)
No data found for 136, 2025, 8.
Failed to get or parse HTML: no text parsed from document (line 0)
No data found for 136, 2025, 9.
Failed to get or parse HTML: no text parsed from document (line 0)
No data found for 136, 2025, 10.
Failed to get or parse HTML: no text parsed from document (line 0)
No data found for 136, 2025, 11.
Failed to get or 